Getting data from web about pincodes of the Bengaluru District

In [4]:
import pandas as pd

In [5]:
import pandas as pd
df=pd.read_html("https://www.mapsofindia.com/pincode/india/karnataka/bangalore/")

In [6]:
len(df)

1

In [7]:
df=df[0]

In [8]:
df.head()

,Pincode Details,Pincode Details.1,Pincode Details.2,Pincode Details.3
0,Location,Pincode,State,District
1,A F station yelahanka,560063,Karnataka,Bangalore
2,Adugodi,560030,Karnataka,Bangalore
3,Agara,560034,Karnataka,Bangalore
4,Agram,560007,Karnataka,Bangalore


In [9]:
df = df.drop([0],axis=0)

In [10]:
df=df.rename(columns={"Pincode Details":"Location","Pincode Details.1":"Pin Code","Pincode Details.2":"State","Pincode Details.3":"District"})

In [11]:
df

,Location,Pin Code,State,District
1,A F station yelahanka,560063,Karnataka,Bangalore
2,Adugodi,560030,Karnataka,Bangalore
3,Agara,560034,Karnataka,Bangalore
4,Agram,560007,Karnataka,Bangalore
5,Air Force hospital,560007,Karnataka,Bangalore
...,...,...,...,...
290,Yelahanka,560064,Karnataka,Bangalore
291,Yelahanka Satellite town,560064,Karnataka,Bangalore
292,Yemalur,560037,Karnataka,Bangalore
293,Yeshwanthpur Bazar,560022,Karnataka,Bangalore


### Getting the latitudes and longitudes

In [29]:
from geopy.geocoders import Nominatim

In [13]:
la=list()
lt=list()
poc=list()

In [14]:
df

,Location,Pin Code,State,District
1,A F station yelahanka,560063,Karnataka,Bangalore
2,Adugodi,560030,Karnataka,Bangalore
3,Agara,560034,Karnataka,Bangalore
4,Agram,560007,Karnataka,Bangalore
5,Air Force hospital,560007,Karnataka,Bangalore
...,...,...,...,...
290,Yelahanka,560064,Karnataka,Bangalore
291,Yelahanka Satellite town,560064,Karnataka,Bangalore
292,Yemalur,560037,Karnataka,Bangalore
293,Yeshwanthpur Bazar,560022,Karnataka,Bangalore


In [32]:
for index , rows in df.iterrows():
    address = rows["Location"] + "," + "Karnataka"
    geolocator = Nominatim(user_agent = "ben_exp")
    location = geolocator.geocode(address)
    try:
        longitude = location.longitude
        latitude = location.latitude
    except:
        longitude = "Not Found"
        latitude = "Not Found"
    poc.append(rows["Pin Code"])
    la.append(latitude)
    lt.append(longitude) 
    

In [ ]:
len(poc)

In [ ]:
df.shape

In [ ]:
df.insert(4,"Pin",poc)
df.insert(5,"Latitude",la)
df.insert(6,"Longitude",lt)

In [ ]:
df=pd.read_csv(r"C:\Users\Ravi\Desktop\projects\benggal)

In [ ]:
df.head()

In [ ]:
df[df["Latitude"]=="Not Found"]


### Dropping the rows without latitude and longitude

In [ ]:
df.drop(df[df["Latitude"]=="Not Found"].index,inplace=True)

In [ ]:
df.reset_index(inplace=True)

In [ ]:
df.drop("index",inplace=True,axis=1)

In [ ]:
df["Pin Code"].unique().size

In [40]:
df.drop("Pin",axis=1,inplace=True)

In [33]:
address= "Bengaluru , Karnataka"
location = geolocator.geocode(address)
Latitude = location.latitude
Longitude= location.longitude

In [42]:
import folium
map_ben= folium.Map(location=[Latitude,Longitude],zoom_start=10)
for lat,lan,loc,pin in zip(df["Latitude"],df["Longitude"],df["Location"],df["Pin Code"]):
    label="{},{}".format(loc,pin)
    label=folium.Popup(label,parse_html=True)
    folium.CircleMarker(
    [lat,lan],
    radius=5,
    popup=label,
    color='black',
    fill_color="#3185cc",
    fill_opactiy=0.7,
    parse_html=False).add_to(map_ben)
map_ben


## Using Foursquare API to get the neighbourhood data

In [1]:
CLIENT_ID = "WC1SOZPQQEG3VFZF0XTG34EBIZFV0KU4IBOQMPBJIHXHMTWS"
CLIENT_SECRET = "EPDWGH4OCFDJDL3QDXJXO3VOLU5OJKNQADNPLPQNZRIWDCG5"
VERSION = "20200816"

In [ ]:
LIMIT = 100
RADIUS = 700
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    Latitude, 
    Longitude, 
    RADIUS, 
    LIMIT)
url 

### Importing the required libraries for handling the data received from the API

In [3]:
import json
import requests 
from pandas import json_normalize
results = requests.get(url).json()
results

### Defining a function to get category type of each venue

In [46]:
def get_category_type (row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row["venue.categories"]
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

### Processing the received information and extracting the required fields

In [47]:
venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues)  # flatten json
# Now filtering coloumns
filtered_columns= ['venue.name', 'venue.categories','venue.location.lat','venue.location.lng']
nearby_venues= nearby_venues.loc[:, filtered_columns]

#filtering the category of each row
nearby_venues["venue.categories"]=nearby_venues.apply(get_category_type,axis= 1)
#clean columns
nearby_venues.columns= [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues.head()

,name,categories,lat,lng
0,Cubbon Park,Park,12.977042,77.595277
1,Vidhana Soudha,Capitol Building,12.979027,77.591881
2,Chevron Hotel,Hotel,12.983619,77.594379
3,Oriental Spice,Chinese Restaurant,12.983893,77.593591
4,Atria Hotel,Hotel,12.980227,77.586067


In [48]:
nearby_venues.shape

(11, 4)

# Part 1 - Analysis of suitable location for joint family

### Defining a function to get the nearyby schools , parks , other dance schools etc for kids

In [5]:
def getNearbyVenues(names,latitudes, longitudes , radius = 500,limit=50):
    venues_list=[]
    for name , lat, lng in zip (names , latitudes , longitudes):
        #creating the url request
        url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId=4bf58dd8d48988d13b941735,4bf58dd8d48988d196941735,4bf58dd8d48988d15a941735'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    lat, 
    lng, 
    radius, 
    limit)
        
        #making the request
        results = requests.get(url).json()['response']
        #getting required informations only
        
        for i in range (len(results["venues"])):
            venues_list.append([name,lat,lng,
                        results['venues'][i]['name'],
                        results["venues"][i]["location"]["lat"],
                        results["venues"][i]['categories'][0]["name"]])
    nearby_venues=pd.DataFrame(venues_list,columns=['location','lat',"long","Venue","Venue Lat","Venue Cat"])
            
    return (nearby_venues)
    

In [6]:
df=pd.read_csv(r"C:\Users\Ravi\Desktop\projects\bngl.csv")

In [7]:
df.head()

,Location,Pin Code,State,District,Latitude,Longitude
0,Adugodi,560030,Karnataka,Bangalore,12.942847,77.610416
1,Agara,560034,Karnataka,Bangalore,13.014409,78.048901
2,Amruthahalli,560092,Karnataka,Bangalore,13.066513,77.596624
3,Anandnagar,560024,Karnataka,Bangalore,13.033377,77.589523
4,Anekal,562106,Karnataka,Bangalore,12.799122,77.680604


In [8]:
bnglu_venues= getNearbyVenues(names = df["Location"],
                             latitudes= df["Latitude"],
                             longitudes = df["Longitude"])

In [10]:
bnglu_venues.to_csv(r"C:\Users\Ravi\Desktop\projects\fam_bngl.csv")

In [11]:
bnglu_venues

,location,lat,long,Venue,Venue Lat,Venue Cat
0,Amruthahalli,13.066513,77.596624,"Amrutha Hospital, Amruthahalli",13.066810,Hospital
1,Anandnagar,13.033377,77.589523,Baptist Hospital,13.035633,Hospital
2,Anandnagar,13.033377,77.589523,Orthopaedic Department Baptist Hospital,13.035852,Hospital
3,Anandnagar,13.033377,77.589523,Otorhinolaryngology department BBH,13.036388,Hospital
4,Anandnagar,13.033377,77.589523,Surau (Musolla) Baptist Hospital,13.035556,Hospital


In [12]:
bnglu_venues.head()


,location,lat,long,Venue,Venue Lat,Venue Cat
0,Amruthahalli,13.066513,77.596624,"Amrutha Hospital, Amruthahalli",13.066810,Hospital
1,Anandnagar,13.033377,77.589523,Baptist Hospital,13.035633,Hospital
2,Anandnagar,13.033377,77.589523,Orthopaedic Department Baptist Hospital,13.035852,Hospital
3,Anandnagar,13.033377,77.589523,Otorhinolaryngology department BBH,13.036388,Hospital
4,Anandnagar,13.033377,77.589523,Surau (Musolla) Baptist Hospital,13.035556,Hospital


In [13]:
bnglu_venues.shape

(489, 6)

In [15]:
bnglu_venues["Venue Cat"].unique()

array(['Hospital', 'High School', 'Middle School', 'School',
       'Nursery School', 'Garden', 'Music School', 'Park', 'Plaza',
       'Pet Service', 'Preschool', 'Language School', 'Elementary School',
       'Emergency Room', 'Driving School', 'Adult Education Center',
       'Religious School', 'Private School', 'Tech Startup'], dtype=object)

### The above venue category type were included for people with family having children and elderly family members. For children included fascilities were schools of all kind , parks and gardens . For elderly people itncludes hospitals and other helath care services and pet care was also included for family pets if they have any

In [22]:
bnglu_venues["location"].value_counts().head(7)

Malleswaram West    21
Malleswaram         21
Vasanthnagar        16
Jayanagar West      14
Madhavan Park       14
Jayanagar           14
Mico Layout         13
Name: location, dtype: int64

In [24]:
bnglu_venues[bnglu_venues["location"] == "Malleswaram"]

,location,lat,long,Venue,Venue Lat,Venue Cat
254,Malleswaram,13.002735,77.570325,Poorvi Sangeeth Academy,13.001936,Music School
255,Malleswaram,13.002735,77.570325,Worldmusic Conservatory,13.001163,School
256,Malleswaram,13.002735,77.570325,Neuro Centre,13.001545,Hospital
257,Malleswaram,13.002735,77.570325,BP Indian,13.000404,Elementary School
258,Malleswaram,13.002735,77.570325,Sanjeevini Hospital,13.004051,Hospital
259,Malleswaram,13.002735,77.570325,Lakshmi Maternity Home,13.005785,Hospital
260,Malleswaram,13.002735,77.570325,Inlingua,13.001080,Language School
261,Malleswaram,13.002735,77.570325,Kidzee Malleswaram,13.002519,Preschool
262,Malleswaram,13.002735,77.570325,Sri Vidya Mandir Education Society,13.002049,High School
263,Malleswaram,13.002735,77.570325,Bangalore Education Socity School,12.998739,High School


### From the above details we can conclude that top 5 Locations having most number of fascilities reqiured for a joint family are :
### 1. Malleswaram
### 2. Vasanthnagar
### 3. Jayanagar
### 4. Madhavan Park
### 5. Mico Layout



# Part 2 - Analysis of locations for Bachelor Job seekers

### Defining a function that will get necessary things for bacelors like food restaurants , dry cleaning services , laundry sevices , nightlife spots etc

In [26]:
def getNearbyVenuesBach(names,latitudes, longitudes , radius = 500,limit=50):
    venues_list=[]
    for name , lat, lng in zip (names , latitudes , longitudes):
        #creating the url request
        url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId=4bf58dd8d48988d1f9941735,4bf58dd8d48988d10f941735,4bf58dd8d48988d1fc941735,52f2ab2ebcbc57f1066b8b1d,4d4b7105d754a06376d81259'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    lat, 
    lng, 
    radius, 
    limit)
        
        #making the request
        results = requests.get(url).json()['response']
        #getting required informations only
        
        for i in range (len(results["venues"])):
            venues_list.append([name,lat,lng,
                        results['venues'][i]['name'],
                        results["venues"][i]["location"]["lat"],
                        results["venues"][i]['categories'][0]["name"]])
    nearby_venues=pd.DataFrame(venues_list,columns=['location','lat',"long","Venue","Venue Lat","Venue Cat"])
            
    return (nearby_venues)
    

In [27]:
bach= getNearbyVenuesBach(names = df["Location"],
                             latitudes= df["Latitude"],
                             longitudes = df["Longitude"])

In [28]:
bach.head()

,location,lat,long,Venue,Venue Lat,Venue Cat
0,Adugodi,12.942847,77.610416,Hd-3,12.943977,Cocktail Bar
1,Adugodi,12.942847,77.610416,Bharathi Refreshments(South Indian Food) - Adu...,12.943388,Fast Food Restaurant
2,Adugodi,12.942847,77.610416,Sri Ganesh Upahaar,12.941766,Indian Restaurant
3,Adugodi,12.942847,77.610416,Navaneetha Andhra Restaurant,12.942357,Andhra Restaurant
4,Adugodi,12.942847,77.610416,adigas,12.940589,Indian Restaurant


In [29]:
bach.to_csv(r"C:\Users\Ravi\Desktop\projects\bngl_bac.csv")

In [30]:
bach.head()

,location,lat,long,Venue,Venue Lat,Venue Cat
0,Adugodi,12.942847,77.610416,Hd-3,12.943977,Cocktail Bar
1,Adugodi,12.942847,77.610416,Bharathi Refreshments(South Indian Food) - Adu...,12.943388,Fast Food Restaurant
2,Adugodi,12.942847,77.610416,Sri Ganesh Upahaar,12.941766,Indian Restaurant
3,Adugodi,12.942847,77.610416,Navaneetha Andhra Restaurant,12.942357,Andhra Restaurant
4,Adugodi,12.942847,77.610416,adigas,12.940589,Indian Restaurant


In [33]:
bach.shape

(2494, 6)

In [46]:
bach["Venue Cat"].unique()

array(['Cocktail Bar', 'Fast Food Restaurant', 'Indian Restaurant',
       'Andhra Restaurant', 'Beer Garden', 'Cafeteria',
       'Food & Drink Shop', 'Karnataka Restaurant', 'Bar', 'Wine Bar',
       'South Indian Restaurant', 'Chaat Place', 'Dhaba', 'Whisky Bar',
       'Other Nightlife', 'Hotel Bar', 'Farmers Market', 'Grocery Store',
       'Laundry Service', 'Liquor Store', 'Udupi Restaurant',
       'Szechuan Restaurant', 'Breakfast Spot', 'North Indian Restaurant',
       'College Cafeteria', 'Multicuisine Indian Restaurant', 'Brewery',
       'Nightclub', 'Fish Market', 'Snack Place', 'Butcher',
       'Punjabi Restaurant', 'Department Store',
       'Maharashtrian Restaurant', 'Vegetarian / Vegan Restaurant',
       'Bakery', 'Indian Sweet Shop', 'Dry Cleaner', 'Pub',
       'Gourmet Shop', 'Kerala Restaurant', 'Indian Chinese Restaurant',
       'Dairy Store', 'Tennis Stadium', 'Sports Bar',
       'Middle Eastern Restaurant', 'Supermarket', 'Diner', 'Gay Bar',
       'Strip

### The above are the categories that are used for analyzing the locations for bachelors who are seeking the job.

#### Now lets check the locations having maximum number of required venues.

In [34]:
bach["location"].value_counts()

Hosur Road             50
Brigade Road           50
Museum Road            50
Hsr Layout             50
Koramangala I block    49
                       ..
Bagalgunte              1
Goripalya               1
Yemalur                 1
Jakkur                  1
Kadabagere              1
Name: location, Length: 122, dtype: int64

In [37]:
bach[bach["location"] == "Hosur Road"].head(10)

,location,lat,long,Venue,Venue Lat,Venue Cat
709,Hosur Road,12.930723,77.614401,Bier Garten,12.933482,Brewery
710,Hosur Road,12.930723,77.614401,Crawl Street,12.932611,Bar
711,Hosur Road,12.930723,77.614401,Hyderabad House,12.932583,Indian Restaurant
712,Hosur Road,12.930723,77.614401,Madeena Hotel,12.935153,Indian Restaurant
713,Hosur Road,12.930723,77.614401,Vijayawada Style,12.933852,Indian Restaurant
714,Hosur Road,12.930723,77.614401,XOOX Brewmill,12.935507,Brewery
715,Hosur Road,12.930723,77.614401,Coin Laundromat,12.926370,Laundry Service
716,Hosur Road,12.930723,77.614401,Rajdhani Snacklet,12.934707,Rajasthani Restaurant
717,Hosur Road,12.930723,77.614401,Idli Dosa Coffee,12.933630,South Indian Restaurant
718,Hosur Road,12.930723,77.614401,Fenny's,12.935101,Bar


## From tha above data we can say that top 5 location for bachelor job seekers or employes are : 
### 1. Hosur Road
### 2. Brigade Road
### 3. Museum Road
### 4. HSR Layout
### 5. Koramangla I Block


# Part 3 Analysis of location for college students

### Defing a function for that are requuired for college students like hostels, stationery shop , library , laundry , book stores , food , cafeteria etc.

In [39]:
def getNearbyVenuesStu(names,latitudes, longitudes , radius = 500,limit=50):
    venues_list=[]
    for name , lat, lng in zip (names , latitudes , longitudes):
        #creating the url request
        url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId=52f2ab2ebcbc57f1066b8b21,4bf58dd8d48988d1fc941735,52f2ab2ebcbc57f1066b8b1d,4bf58dd8d48988d1ee931735,4d4b7105d754a06372d81259,4bf58dd8d48988d114951735'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    lat, 
    lng, 
    radius, 
    limit)
        
        #making the request
        results = requests.get(url).json()['response']
        #getting required informations only
        
        for i in range (len(results["venues"])):
            venues_list.append([name,lat,lng,
                        results['venues'][i]['name'],
                        results["venues"][i]["location"]["lat"],
                        results["venues"][i]['categories'][0]["name"]])
    nearby_venues=pd.DataFrame(venues_list,columns=['location','lat',"long","Venue","Venue Lat","Venue Cat"])
            
    return (nearby_venues)
    

In [40]:
students = getNearbyVenuesStu(names = df["Location"],
                             latitudes= df["Latitude"],
                             longitudes = df["Longitude"])

In [41]:
students.to_csv(r"C:\Users\Ravi\Desktop\projects\students.csv")

In [42]:
students.head()

,location,lat,long,Venue,Venue Lat,Venue Cat
0,Adugodi,12.942847,77.610416,Indian Retail School,12.942211,College Administrative Building
1,Adugodi,12.942847,77.610416,Evergreen Paying Guest,12.939677,Hostel
2,Adugodi,12.942847,77.610416,Sysinnova Infotech,12.944200,Student Center
3,Adugodi,12.942847,77.610416,National institute of Animal Nutrition and Phy...,12.943050,College Auditorium
4,Amruthahalli,13.066513,77.596624,Bright Kid Montessori House,13.069784,College Academic Building


In [44]:
students.shape

(1121, 6)

In [45]:
students["Venue Cat"].unique()

array(['College Administrative Building', 'Hostel', 'Student Center',
       'College Auditorium', 'College Academic Building',
       'College Library', 'College Classroom', 'University',
       'College Engineering Building', 'Medical School',
       'College Rec Center', 'College Cafeteria',
       'General College & University', 'College Technology Building',
       'Laundry Service', 'Bookstore', 'College Bookstore', 'College Gym',
       'College Residence Hall', 'Law School',
       'College Communications Building', 'Fraternity House',
       'Trade School', 'College Science Building',
       'College Basketball Court', 'College Lab', 'College Math Building',
       'Dry Cleaner', 'College Theater', 'College Arts Building',
       'College & University', 'College Soccer Field', 'College Quad',
       'Community College', 'Stationery Store', 'College Football Field',
       'College Hockey Rink', 'College Cricket Pitch', 'Hospital',
       'College Tennis Court', 'Hotel', 'Café'

### The above venue category are used to analyze the locations for college students

### Lets check locations having maximum of these venues.

In [47]:
students["location"].value_counts()

Msrit                50
Dharmaram College    47
Taverekere           47
Brigade Road         46
Museum Road          44
                     ..
Deepanjalinagar       1
Hulimavu              1
Nayandahalli          1
Ullalu Upanagara      1
Viveknagar            1
Name: location, Length: 117, dtype: int64

In [49]:
students[students["location"]== "Msrit"].head(10)

,location,lat,long,Venue,Venue Lat,Venue Cat
736,Msrit,13.030955,77.564856,M S Ramaiah Institute of Technology Bangalore,13.031549,General College & University
737,Msrit,13.030955,77.564856,New MSRIT canteen,13.031370,College Cafeteria
738,Msrit,13.030955,77.564856,MSRIT,13.031312,College & University
739,Msrit,13.030955,77.564856,M S Ramaiah medical college,13.030790,Medical School
740,Msrit,13.030955,77.564856,"JRD Tata Auditorium, NIAS",13.029984,College Auditorium
741,Msrit,13.030955,77.564856,Msr Law Building,13.030105,Law School
742,Msrit,13.030955,77.564856,svit Bangalore,13.031368,College Academic Building
743,Msrit,13.030955,77.564856,NIAS,13.029176,College Engineering Building
744,Msrit,13.030955,77.564856,M.S.Ramaiah Engineering Boys Hostel,13.032142,College Residence Hall
745,Msrit,13.030955,77.564856,laundry,13.033875,Laundry Service


## From the above data we can say that top 5 locations for college students are:
### 1. Msrit
### 2. Dharmaram College
### 3. Taverekere
### 4. Brigade Road
### 5. Museum Road






#                            PROJECT COMPLETE !!